In [548]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [549]:
vod_08 = pd.read_csv('../data/데이터스쿨3차_2308월/데이터스쿨_3차_VOD_2308.csv', encoding = 'cp949', sep = '\t')
vod_09 = pd.read_csv('../data/데이터스쿨3차_2309월/데이터스쿨_3차_VOD_2309.csv', encoding = 'cp949', sep = '\t')
vod_10 = pd.read_csv('../data/데이터스쿨3차_VOD_10월.csv', encoding= 'cp949')

vod_all = pd.concat([vod_08, vod_09, vod_10], ignore_index=True)
print(vod_all.shape)
vod_all.head()

(15219, 9)


,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230812163507
1,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230816205227
2,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,718,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230829194727
3,61689000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회(22/07/21),TV 시사/교양,기타,4183,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",1:13,20230804092737


In [550]:
def preprocessing(data):
  df = data.copy()
  
  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    # 입력값이 NaN이면 0 반환
    if pd.isna(runtime_str):
        return 0
    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  # 런타임 0분인 것 제거
  df = df[df['disp_rtm'] != 0]

  # 무삭제 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'무삭제판|무삭제', '', regex = True)
  
  # 예고편 제거
  df = df[~df['asset_nm'].str.contains(r'\(예고편\)|\(예고\)', regex=True)]
  
  # 예약구매, 사전구매 제거
  df = df[~df['asset_nm'].str.contains('예약구매|사전구매')]

  # 00회가 포함된 행은 런타임이 0또는 1이므로 제거
  df = df[~df['asset_nm'].str.contains(r'\b00회\b')]
  
  ## 괄호와 그 안의 내용 제거 
  df['asset_nm'] = df['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\…', '', regex=True) # … 기호 제거
  df['asset_nm'] = df['asset_nm'].str.replace('-', " ")
  df['asset_nm'] = df['asset_nm'].str.rstrip('.')

  # use_tms 분 단위로 변경
  df['use_tms'] = round(df['use_tms'] / 60, 1)

  # 셋탑번호가 66056000인 것 삭제 - content 에서 이상치 아이디로 판별
  df = df[df['subsr'] != 66056000]

  return df

In [551]:
df = preprocessing(vod_all)
df.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737


In [552]:
# 프로그램 회차별 정보
df1 = df[['asset_nm', 'ct_cl', 'genre_of_ct_cl', 'ACTR_DISP', 'disp_rtm']].drop_duplicates().reset_index(drop = True)
df1.head()

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다 1361회,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기 61회,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장 3338회,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기 89회,TV 시사/교양,기타,"장도연,장현성,장성규",78


In [553]:
# 유저마다 각 회차를 본 시간을 더함
df2 = pd.DataFrame(df.groupby(['subsr','asset_nm', 'ct_cl', 'genre_of_ct_cl', 'disp_rtm'])['use_tms'].sum()).reset_index()
df2 = df2[(df2['disp_rtm'] != 0) & (df2['use_tms'] != 0)]
df2

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms
0,59879000,소방서 옆 경찰서 05회,TV드라마,기타,69,2.4
1,59879000,소방서 옆 경찰서 06회,TV드라마,기타,59,22.1
2,59879000,신성한 이혼 01회,TV드라마,기타,64,12.6
3,59879000,신성한 이혼 02회,TV드라마,기타,62,16.8
4,59879000,신성한 이혼 03회,TV드라마,기타,63,4.3
...,...,...,...,...,...,...
8012,67170000,빨간 모자,키즈,기타,5,5.0
8013,67170000,태극기휘날리며,영화,액션/어드벤쳐,147,0.5
8014,67174000,가문의 영광: 리턴즈,영화,코미디,99,1.4
8015,67202000,검객,영화,액션/어드벤쳐,100,26.8


In [554]:
# 시청시간으로 유저가 이 회차를 시청했는지 여부를 결정
## 시청시간을 런타임으로 나눠 0.6 이 넘으면 시청한 것으로 간주
df2['watched'] = df2['use_tms'] / df2['disp_rtm']
df2['watched'] = df2['watched'].apply(lambda x : 1 if x >= 0.6 else 0)
df2.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms,watched
0,59879000,소방서 옆 경찰서 05회,TV드라마,기타,69,2.4,0
1,59879000,소방서 옆 경찰서 06회,TV드라마,기타,59,22.1,0
2,59879000,신성한 이혼 01회,TV드라마,기타,64,12.6,0
3,59879000,신성한 이혼 02회,TV드라마,기타,62,16.8,0
4,59879000,신성한 이혼 03회,TV드라마,기타,63,4.3,0


In [555]:
# 회차 제거
df11 = df1.copy()

df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회\.', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+화$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.strip()
df11['asset_nm'] = df11['asset_nm'].str.rstrip('.')
df11

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",78
...,...,...,...,...,...
5207,핑크퐁 호기의 우주동요,키즈,학습,"핑크퐁,호기",1
5208,더 놀다 잘래요,키즈,학습,-,9
5209,핑크퐁 호기의 우주동요,키즈,학습,"핑크퐁,호기",1
5210,달샤베트,키즈,학습,-,5


In [556]:
# 유저 시청 정보의 회차 제거한 변수 추가
df2['series_nm'] = df2['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.replace(r'\d+회\.', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.replace(r'\d+화$', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.strip()
df2['series_nm'] = df2['series_nm'].str.rstrip('.')
df2.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms,watched,series_nm
0,59879000,소방서 옆 경찰서 05회,TV드라마,기타,69,2.4,0,소방서 옆 경찰서
1,59879000,소방서 옆 경찰서 06회,TV드라마,기타,59,22.1,0,소방서 옆 경찰서
2,59879000,신성한 이혼 01회,TV드라마,기타,64,12.6,0,신성한 이혼
3,59879000,신성한 이혼 02회,TV드라마,기타,62,16.8,0,신성한 이혼
4,59879000,신성한 이혼 03회,TV드라마,기타,63,4.3,0,신성한 이혼


In [557]:
df3 = df2[['subsr', 'series_nm', 'ct_cl',	'genre_of_ct_cl', 'watched']].copy()
df3.head()

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched
0,59879000,소방서 옆 경찰서,TV드라마,기타,0
1,59879000,소방서 옆 경찰서,TV드라마,기타,0
2,59879000,신성한 이혼,TV드라마,기타,0
3,59879000,신성한 이혼,TV드라마,기타,0
4,59879000,신성한 이혼,TV드라마,기타,0


In [558]:
# 유저의 시리즈별 시청 횟수
df4 = df3.groupby(['subsr', 'series_nm', 'ct_cl', 'genre_of_ct_cl'])['watched'].sum().reset_index()
df4 = df4[df4['watched']!= 0]
df4

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched
2,59895000,금이야 옥이야,TV드라마,기타,1
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,1
4,59900000,7인의 탈출,TV드라마,기타,1
5,59900000,그것이알고싶다,TV 시사/교양,기타,3
10,59900000,범죄도시3,영화,액션/어드벤쳐,1
...,...,...,...,...,...
2915,67170000,건축학개론,영화,멜로,1
2916,67170000,고딩엄빠 2,TV 연예/오락,기타,3
2917,67170000,말죽거리잔혹사,영화,액션/어드벤쳐,1
2918,67170000,빨간 모자,키즈,기타,1


In [559]:
# 8~9월 시리즈별 시청된 총 횟수
df5 = pd.DataFrame(df11[['asset_nm', 'ct_cl', 'genre_of_ct_cl']].value_counts().reset_index())
df5.columns = ['series_nm', 'ct_cl', 'genre_of_ct_cl', 'watched_all']
df5

,series_nm,ct_cl,genre_of_ct_cl,watched_all
0,금이야 옥이야,TV드라마,기타,116
1,연희공략: 건륭황제의여인,TV드라마,외화 시리즈,70
2,인간극장,TV 시사/교양,기타,68
3,TV소설 은희,TV드라마,기타,63
4,런닝맨,TV 연예/오락,기타,58
...,...,...,...,...
1392,범죄도시2,영화,액션/어드벤쳐,1
1393,범죄도시 2,영화,액션/어드벤쳐,1
1394,버섯도리TV 버섯탐정 시즌2,TV애니메이션,기타,1
1395,버섯도리: 패밀리 대작전,TV애니메이션,기타,1


In [560]:
df6 = df4.merge(df5, on = ['series_nm', 'ct_cl', 'genre_of_ct_cl'], how = 'left')
df6

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all
0,59895000,금이야 옥이야,TV드라마,기타,1,116
1,59900000,2022 역사저널 그날,TV 시사/교양,기타,1,3
2,59900000,7인의 탈출,TV드라마,기타,1,12
3,59900000,그것이알고싶다,TV 시사/교양,기타,3,28
4,59900000,범죄도시3,영화,액션/어드벤쳐,1,1
...,...,...,...,...,...,...
1682,67170000,건축학개론,영화,멜로,1,1
1683,67170000,고딩엄빠 2,TV 연예/오락,기타,3,5
1684,67170000,말죽거리잔혹사,영화,액션/어드벤쳐,1,1
1685,67170000,빨간 모자,키즈,기타,1,1


In [561]:
# 유저의 프로그램 총 시청 횟수
df7 = df6.groupby(['subsr'])['watched'].sum().reset_index()
df7.columns = ['subsr', 'watched_cnt']
df7

,subsr,watched_cnt
0,59895000,1
1,59900000,7
2,59930000,12
3,59933000,12
4,60040000,1
...,...,...
341,67140000,3
342,67148000,10
343,67164000,4
344,67170000,6


In [562]:
df8 = df6.merge(df7, on = 'subsr', how = 'left')
df8 = df8[df8['watched_cnt'] != 0].reset_index(drop = True)
df8

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all,watched_cnt
0,59895000,금이야 옥이야,TV드라마,기타,1,116,1
1,59900000,2022 역사저널 그날,TV 시사/교양,기타,1,3,7
2,59900000,7인의 탈출,TV드라마,기타,1,12,7
3,59900000,그것이알고싶다,TV 시사/교양,기타,3,28,7
4,59900000,범죄도시3,영화,액션/어드벤쳐,1,1,7
...,...,...,...,...,...,...,...
1682,67170000,건축학개론,영화,멜로,1,1,6
1683,67170000,고딩엄빠 2,TV 연예/오락,기타,3,5,6
1684,67170000,말죽거리잔혹사,영화,액션/어드벤쳐,1,1,6
1685,67170000,빨간 모자,키즈,기타,1,1,6


In [563]:
# 시청횟수가 5개 이상인 유저(heavy users)
# 시청횟수가 3~4개인 유저(medium users)
# 시청횟수가 1~2개인 유저(light users) 
user_cnt = df8['subsr'].value_counts()
heavy_users = user_cnt[user_cnt >= 5].index
medium_users = user_cnt[user_cnt >= 3].index
light_users = user_cnt[user_cnt >= 1].index

vod_heavy = df8[df8['subsr'].isin(heavy_users)]
vod_medium = df8[df8['subsr'].isin(medium_users)]
vod_light = df8[df8['subsr'].isin(light_users)]

In [564]:
# 시청 여부, 해당 시리즈 총 횟수, 유저의 프로그램 총 시청 횟수를 이용해
# 유저의 프로그램 선호도를 측정
import numpy as np
def scoring(df):
  N = df['watched_all'] # 해당 프로그램 전체 회차 수
  L = df['watched_cnt'] # 유저의 총 시청 프로그램 회차 수
  n = df['watched'] # 유저의 해당 프로그램 시청 수
  lam = np.log(2) / 2
  w1 = 1 - np.exp(-1 * lam * N)
  data = pd.DataFrame([N, L]).T
  def custom_weight(data):
    if data['watched_cnt'] < data['watched_all']:
      return data['watched_all'] / data['watched_cnt']
    else:
      return 1

  w2 = data.apply(custom_weight, axis = 1)

  score = (n / N) * w1 * w2

  return score

In [565]:
vod_heavy['score'] = scoring(vod_heavy) 
vod_medium['score'] = scoring(vod_medium)
vod_light['score'] = scoring(vod_light)

In [566]:
def add_cat_rename(vod_data):
    vod = vod_data[['subsr', 'series_nm', 'ct_cl', 'genre_of_ct_cl', 'score']].copy()
    vod = vod.sort_values(by = 'subsr').reset_index(drop = True)
    vod['category'] = vod['ct_cl'].apply(lambda x : x if x in ['영화', '키즈'] else 'TV프로그램')
    vod['rename'] = vod['series_nm'].apply(lambda x : x.replace(' ', '') if isinstance(x, str) else x)

    return vod

vod_heavy = add_cat_rename(vod_heavy)
vod_medium = add_cat_rename(vod_medium)
vod_light = add_cat_rename(vod_light)

In [567]:
vod_id = pd.read_csv('../data/vod_list_add10_1220.csv', index_col=0)
vod_id = vod_id[['rename', 'Category', 'vod_id']]
vod_id.columns = ['rename', 'category', 'vod_id']

vod_heavy_id = vod_heavy.merge(vod_id, on = ['rename', 'category'], how = 'left')[['subsr', 'vod_id', 'score']]
vod_medium_id = vod_medium.merge(vod_id, on = ['rename', 'category'], how = 'left')[['subsr', 'vod_id', 'score']]
vod_light_id = vod_light.merge(vod_id, on = ['rename', 'category'], how = 'left')[['subsr', 'vod_id', 'score']]

In [568]:
vod_heavy_id.to_csv('../data/vod_heavy_score.csv', index = 0)
vod_medium_id.to_csv('../data/vod_medium_score.csv', index = 0)
vod_light_id.to_csv('../data/vod_light_score.csv', index = 0)

In [569]:
vod_heavy_id = pd.read_csv('../data/vod_heavy_score.csv')
vod_medium_id = pd.read_csv('../data/vod_medium_score.csv')
vod_light_id = pd.read_csv('../data/vod_light_score.csv')

In [570]:
# 필요한 Surprise 알고리즘 불러오기
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline, NMF, KNNWithMeans, KNNBasic
from surprise import accuracy
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# surprise 데이터 형식으로 변환
def convert_traintest_dataframe_forsurprise(training_dataframe):
    reader = Reader(rating_scale=(0, 1)) # 이 범위를 넘으면 양극값으로 대체
    trainset = Dataset.load_from_df(training_dataframe[['subsr', 'vod_id', 'score']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    return trainset

trainset = convert_traintest_dataframe_forsurprise(vod_heavy_id)

In [571]:
sim_options = {'name': 'pearson_baseline', 'user_based': False} # item-based similarity
bsl_options = {'method' : 'sgd', 'n_epochs' : 1}
knnbaseline = KNNBaseline(k = 40, sim_options=sim_options, random_state = 42, min_k= 1, 
                          bsl_options=bsl_options)

knnbaseline.fit(trainset)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [572]:
user_id = sorted(vod_heavy_id.subsr.unique())
vod_id = sorted(vod_heavy_id.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59900000,3,0.348355
1,59900000,6,0.348973
2,59900000,7,0.348701
3,59900000,8,0.431166
4,59900000,12,0.347568
...,...,...,...
68197,67055000,5014,0.345379
68198,67055000,5015,0.345314
68199,67055000,5016,0.345380
68200,67055000,5018,0.345335


In [573]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:25].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,59900000,111,14,3028,16,542,292,289,143,44,...,245,1156,1347,2979,888,848,4833,398,887,1395
1,59930000,728,456,1156,1347,2979,888,848,111,4833,...,719,4752,880,1077,830,4776,858,754,114,530
2,60050000,213,360,1156,1347,2979,888,848,111,4833,...,719,4752,880,1077,830,4776,858,754,114,530
3,60067000,954,4782,992,976,1347,849,1136,2404,810,...,4763,4833,1156,888,848,111,398,1395,720,726
4,60224000,410,4787,1886,835,1166,256,508,1278,1178,...,2979,888,848,111,4833,398,887,1395,720,726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,66838000,1156,1347,2979,888,848,111,4833,398,887,...,880,1077,830,4776,858,754,114,530,356,4784
77,66873000,1156,738,490,531,784,1886,835,842,1278,...,508,3028,293,1347,2979,848,4833,398,887,1395
78,66875000,4776,4906,1347,4825,4814,731,4233,4790,357,...,3502,1052,987,1131,3457,4777,117,4766,1133,4824
79,66900000,848,1156,1347,2979,888,4833,398,887,1395,...,1077,830,4776,858,754,114,530,356,4784,816


In [574]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59900000,"[111, 14, 3028, 16, 542, 292, 289, 143, 44, 29..."
1,59930000,"[728, 456, 1156, 1347, 2979, 888, 848, 111, 48..."
2,60050000,"[213, 360, 1156, 1347, 2979, 888, 848, 111, 48..."
3,60067000,"[954, 4782, 992, 976, 1347, 849, 1136, 2404, 8..."
4,60224000,"[410, 4787, 1886, 835, 1166, 256, 508, 1278, 1..."
...,...,...
76,66838000,"[1156, 1347, 2979, 888, 848, 111, 4833, 398, 8..."
77,66873000,"[1156, 738, 490, 531, 784, 1886, 835, 842, 127..."
78,66875000,"[4776, 4906, 1347, 4825, 4814, 731, 4233, 4790..."
79,66900000,"[848, 1156, 1347, 2979, 888, 4833, 398, 887, 1..."


In [575]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
vod_list = pd.read_csv('../data/vod_list_add10_1220.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = vod_heavy_id[vod_heavy_id.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue
    
    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
result_vod_heavy = vod_predict_1.copy()

In [576]:
trainset = convert_traintest_dataframe_forsurprise(vod_medium_id)

In [577]:
sim_options = {'name': 'pearson_baseline', 'user_based': False} # item-based similarity
bsl_options = {'method' : 'sgd', 'n_epochs' : 1}
knnbaseline = KNNBaseline(k = 40, sim_options=sim_options, random_state = 42, min_k= 1, 
                          bsl_options=bsl_options)

knnbaseline.fit(trainset)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [578]:
user_id = sorted(vod_medium_id.subsr.unique())
vod_id = sorted(vod_medium_id.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59900000,3,0.347783
1,59900000,6,0.348408
2,59900000,7,0.347852
3,59900000,8,0.436353
4,59900000,12,0.346980
...,...,...,...
127535,67170000,5014,0.350412
127536,67170000,5015,0.350349
127537,67170000,5016,0.350413
127538,67170000,5018,0.350344


In [579]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:25].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,59900000,111,14,3028,531,542,143,16,292,289,...,38,245,1156,1347,848,1234,2979,888,973,4833
1,59930000,728,456,1156,1347,848,1234,2979,888,973,...,739,4906,719,754,726,4752,880,43,32,1395
2,60050000,213,360,1156,1347,848,1234,2979,888,973,...,4906,739,719,754,726,4752,880,43,32,1395
3,60067000,954,4782,992,976,1347,849,1136,2404,810,...,4763,4833,1156,848,1234,888,973,111,398,1077
4,60169000,1156,1347,848,1234,2979,888,973,111,4833,...,719,754,726,4752,43,880,32,1395,830,858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,67008000,1156,1347,848,1234,2979,888,973,111,4833,...,719,754,726,4752,880,43,32,1395,830,858
136,67055000,315,4762,490,1156,1347,848,1234,2979,888,...,4906,739,719,754,726,4752,880,43,32,1395
137,67140000,1156,1347,848,1234,2979,888,973,111,4833,...,719,754,726,4752,880,43,32,1395,830,858
138,67164000,1156,1347,848,1234,2979,888,973,111,4833,...,719,754,726,4752,43,880,32,1395,830,858


In [580]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59900000,"[111, 14, 3028, 531, 542, 143, 16, 292, 289, 2..."
1,59930000,"[728, 456, 1156, 1347, 848, 1234, 2979, 888, 9..."
2,60050000,"[213, 360, 1156, 1347, 848, 1234, 2979, 888, 9..."
3,60067000,"[954, 4782, 992, 976, 1347, 849, 1136, 2404, 8..."
4,60169000,"[1156, 1347, 848, 1234, 2979, 888, 973, 111, 4..."
...,...,...
135,67008000,"[1156, 1347, 848, 1234, 2979, 888, 973, 111, 4..."
136,67055000,"[315, 4762, 490, 1156, 1347, 848, 1234, 2979, ..."
137,67140000,"[1156, 1347, 848, 1234, 2979, 888, 973, 111, 4..."
138,67164000,"[1156, 1347, 848, 1234, 2979, 888, 973, 111, 4..."


In [581]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
vod_list = pd.read_csv('../data/vod_list_add10_1220.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = vod_medium_id[vod_medium_id.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue
    
    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
result_vod_medium = vod_predict_1.copy()

In [582]:
trainset = convert_traintest_dataframe_forsurprise(vod_light_id)

In [583]:
sim_options = {'name': 'pearson_baseline', 'user_based': False} # item-based similarity
bsl_options = {'method' : 'sgd', 'n_epochs' : 1}
knnbaseline = KNNBaseline(k = 40, sim_options=sim_options, random_state = 42, min_k= 1, 
                          bsl_options=bsl_options)

knnbaseline.fit(trainset)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [584]:
user_id = sorted(vod_light_id.subsr.unique())
vod_id = sorted(vod_light_id.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59895000,3,0.382139
1,59895000,6,0.382473
2,59895000,7,0.380364
3,59895000,8,0.395019
4,59895000,10,0.382477
...,...,...,...
333193,67204000,5014,0.379410
333194,67204000,5015,0.379361
333195,67204000,5016,0.379412
333196,67204000,5018,0.379354


In [585]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:25].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,59895000,738,784,1156,747,107,842,1178,296,1166,...,401,848,2979,4787,41,973,405,818,368,554
1,59900000,296,143,8,111,16,14,542,531,292,...,297,1166,880,1156,888,1347,1234,401,848,2979
2,59930000,728,456,296,1166,143,880,8,1156,888,...,41,973,405,818,368,554,553,398,4833,887
3,59933000,296,143,8,16,835,531,292,293,861,...,391,289,36,432,448,38,44,297,1166,880
4,60040000,296,1166,143,880,8,1156,888,1347,1234,...,405,818,368,554,553,398,4833,887,1077,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,67140000,296,1166,143,880,8,1156,888,1347,1234,...,405,818,368,554,553,398,4833,887,1077,111
342,67148000,880,296,1166,143,8,1156,888,1347,1234,...,405,818,368,554,553,398,4833,887,1077,111
343,67164000,296,1166,143,880,8,1156,888,1347,1234,...,405,818,368,554,553,398,4833,887,1077,111
344,67170000,948,296,1166,143,880,8,1156,888,1347,...,973,405,818,368,554,553,398,4833,887,1077


In [586]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59895000,"[738, 784, 1156, 747, 107, 842, 1178, 296, 116..."
1,59900000,"[296, 143, 8, 111, 16, 14, 542, 531, 292, 451,..."
2,59930000,"[728, 456, 296, 1166, 143, 880, 8, 1156, 888, ..."
3,59933000,"[296, 143, 8, 16, 835, 531, 292, 293, 861, 72,..."
4,60040000,"[296, 1166, 143, 880, 8, 1156, 888, 1347, 1234..."
...,...,...
341,67140000,"[296, 1166, 143, 880, 8, 1156, 888, 1347, 1234..."
342,67148000,"[880, 296, 1166, 143, 8, 1156, 888, 1347, 1234..."
343,67164000,"[296, 1166, 143, 880, 8, 1156, 888, 1347, 1234..."
344,67170000,"[948, 296, 1166, 143, 880, 8, 1156, 888, 1347,..."


In [587]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
vod_list = pd.read_csv('../data/vod_list_add10_1220.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = vod_light_id[vod_light_id.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue
    
    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
result_vod_light = vod_predict_1.copy()

In [588]:
result_content = pd.read_csv('../data/semi_con_ensemble.csv')

In [589]:
# medium 유저만 뽑아서 추천 결과 생성
medium_users = set(result_vod_medium.subsr.values) - set(result_vod_heavy.subsr.values)
predict_vod_medium = result_vod_medium[result_vod_medium.subsr.isin(medium_users)].reset_index(drop = True)

In [590]:
# light 유저만 뽑아서 추천 결과 생성
## vod 1~2개 본 유저 - (1)
light_users = set(result_vod_light.subsr.values) - set(result_vod_medium.subsr.values)
predict_vod_light = result_vod_light[result_vod_light.subsr.isin(light_users)].reset_index(drop = True)

## vod에는 없고 content 데이터에서 클릭만 한 유저 - (2)
content_user_only = set(result_content.subsr.values) - set(result_vod_light.subsr.values)
result_content_only = result_content[result_content.subsr.isin(content_user_only)]
result_content_only.index = result_content_only.subsr
result_content_only = result_content_only.apply(lambda x : x[1:].tolist(), axis = 1).reset_index()
result_content_only.columns = ['subsr', 'vod_id']

## (1)과 (2)를 결합한 light 유저로 추천 결과 생성
predict_light = pd.concat([result_content_only, predict_vod_light]).reset_index(drop = True)

In [591]:
predict_all = pd.concat([result_vod_heavy, predict_vod_medium, predict_light])

for i in range(10):
    predict_all[f'rec{i}'] = predict_all.vod_id.apply(lambda x:x[i])

predict_all = predict_all.drop(['vod_id'], axis = 1)
predict_all

,subsr,rec0,rec1,rec2,rec3,rec4,rec5,rec6,rec7,rec8,rec9
0,59900000,111.0,14.0,3028.0,16.0,542.0,292.0,289.0,143.0,44.0,296.0
1,59930000,728.0,456.0,1156.0,1347.0,2979.0,888.0,848.0,111.0,4833.0,398.0
2,60050000,213.0,360.0,1156.0,1347.0,2979.0,888.0,848.0,111.0,4833.0,398.0
3,60067000,954.0,4782.0,992.0,976.0,1347.0,849.0,1136.0,2404.0,810.0,884.0
4,60224000,410.0,4787.0,1886.0,835.0,1166.0,256.0,508.0,1278.0,1178.0,747.0
...,...,...,...,...,...,...,...,...,...,...,...
786,67107000,296.0,1166.0,143.0,880.0,8.0,1156.0,888.0,1347.0,1234.0,401.0
787,67117000,818.0,296.0,1166.0,143.0,880.0,8.0,1156.0,888.0,1347.0,1234.0
788,67133000,296.0,143.0,8.0,16.0,835.0,531.0,292.0,293.0,861.0,72.0
789,67148000,880.0,296.0,1166.0,143.0,8.0,1156.0,888.0,1347.0,1234.0,401.0


In [592]:
predict_all.to_csv('../data/recommend_all_user.csv', index = 0)